## Self-training

The execution of this notebook denotes one iteration in the self-training process.

The oracle is assumed to be already trained an stored in `models/oracle`.

Firstly a u-net is trained using the supervised data in the first iteration.

The trained segmenter computes segmentation for unsupervised data. 

The oracle determines the highly confident predictions (either in class 5 or 0 in the `buckets_classification` modality).

A sampling using the size of the computed liver is performed. All selected samples are inserted into the training data (see last line).

Re-execute the notebook to carry out a new iteration.

In [ ]:
import h5py, random, json
import tensorflow as tf
import numpy as np
from libs.generators.utils import get_case_length, get_x_slice, get_y_slice
from libs.models.u_net import get_model_unet
from libs.postprocessing import self_ensembling
from libs.generators.utils import get_x_case, get_y_case

## Initialize model and batch generator

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [ ]:
model = get_model_unet(input_size = (416, 416, 1), feature_maps = 16, output_layers=1, output_type='sigmoid')

In [ ]:
from libs.generators.ssl_batch_generator import SemiSupervisedBatchGenerator

gen = SemiSupervisedBatchGenerator()

## Training

In [ ]:
from keras.optimizers import Adam
from libs.metrics import dice_coef_sig

model.compile(optimizer= Adam(lr=1e-4, clipnorm=1., clipvalue=0.5), loss='binary_crossentropy', metrics=[ dice_coef_sig, 'binary_accuracy'])

In [ ]:
model.fit_generator(gen, epochs = 120, verbose= 1, workers = 32, max_queue_size=100)

In [ ]:
model.compile(optimizer= Adam(lr=1e-5, clipnorm=1., clipvalue=0.5), loss='binary_crossentropy', metrics=[dice_coef_sig, 'binary_accuracy'])

In [ ]:
model.fit_generator(gen, epochs = 30, verbose= 1, workers = 32, max_queue_size=100)

In [ ]:
from libs.keras_checkpoints import load_model, save_model

oracle = load_model('models/oracle')

## Inference and collection of highly confident data by the oracle

In [ ]:
def get_unsupervised_samples(supervised= 0):
    '''Getter of unsupervised data indices.
    @param supervised The id of the supervised volume
    @return indices of unsupervised data
    '''
    indices = []
    with h5py.File('data/training_data.h5', 'r') as hdf:
        for i in range(0, 20):
            if i == supervised:
                continue
            l = get_case_length(hdf, i)
            for j in range(l):
                indices.append((i, j))
        return indices

In [ ]:
indices = get_unsupervised_samples()

In [ ]:
def batch_normalization(x):
    return (x - x.mean()) / x.std()

def stack_segmentation_rgb(x, y):
    b = np.full(x.shape, x.mean(), 'float32')
    g = x + y.reshape(416, 416, 1)
    return batch_normalization(np.concatenate((x, g, b), axis= -1))

In [ ]:
collected_high_iou = []
collected_undefined_iou = []

with h5py.File('data/training_data.h5', 'r') as hdf:
    print('Start evaluating unlabeled data')
    for index in indices:
        x = get_x_slice(hdf, index[0], index[1])
        P = self_ensembling(x, model)
        rgb = stack_segmentation_rgb(x, P)
        oracle_verdict = np.argmax(oracle.predict(rgb.reshape(1, 416, 416, 3))[0])
        if oracle_verdict == 5: # in P_{high}
            # print('Axial slice %i of patient %i is classified as confident prediction.'%(index[0], index[1]))
            collected_high_iou.append((index[0], index[1], P, int(P.sum())))
        if oracle_verdict == 0 and P.sum() == 0: # in P_{nan}
            # print('Axial slice %i of patient %i is classified in the NaN class.'%(index[0], index[1]))
            collected_undefined_iou.append((index[0], index[1], P, int(P.sum())))

## Sampling using the size of the predicted livers

In [ ]:
def sampling(collected_data, generator, liver_area = 35000):
    tau = 5 # min data to add at each iteration
    
    Pn = 0
    while Pn < tau and 0 < liver_area :
        liver_area = liver_area - 1000
        sampled = filter(lambda x: x[3] > tau, collected_data)
        Pn = generator.count_new_samples(sampled)
    return sampled;

In [ ]:
high_confident_data = sampling(collected_high_iou, gen)
random.shuffle(collected_undefined_iou)
sampled_nan = collected_undefined_iou[:len(high_confident_data)]

## Rebuild the semi-supervised dataset and add new data

In [ ]:
from libs.init_ssl_database import init_ssl_database

gen.close_files()
init_ssl_database()
gen = SemiSupervisedBatchGenerator()

In [ ]:
gen.add_separate_samples(high_confident_data, sampled_nan)

## Restart the process